In [2]:
import pandas as pd
import os

# 🚿 Filtering column names

### 📒 Reading dataframes and filtering columns

In [3]:
base_path = './datasets/'
file_name_convention = 'visit'
extension = '.tsv'

# Load the data, scanning in the base path, but load files as needed, not all at once

def get_visit_records():

    filenames = []

    for root, dirs, files in os.walk(base_path):

        for filename in files:
            if filename.startswith(file_name_convention) and filename.endswith(extension):
                filenames.append(filename)
        
    for filename in filenames:
        df = pd.read_csv(
            base_path + filename,
            sep='\t',
            index_col=False
        )
        yield df


In [4]:
# Load variable names

variables = []

with open('./lista_variables/variables_selectas.txt', ) as file:
    variables = [var[:-1] for var in file.readlines()]

print(variables)

['ABBLEED', 'AGE1', 'AGE10', 'AGE11', 'AGE12', 'AGE2', 'AGE3', 'AGE4', 'AGE5', 'AGE6', 'AGE7', 'AGE8', 'AGE9', 'AGE', 'ALCHL24', 'WORSE', 'FOODPNA', 'AVCIGDA', 'GLASBEE', 'GLASLIQ', 'GLASWIN', 'BONES1', 'BONES2', 'BONES3', 'BOTHOTF', 'LEKBOTH', 'COMBIN1', 'COMBIN2', 'NOREMEB', 'BATCARB', 'DTTDFIB', 'BATKCAL', 'DTTALCH', 'BATNIAC', 'BATPHOS', 'BATPOTS', 'BATPROT', 'BATRIBO', 'BATTFAT', 'HOMEXPD', 'DIABETE', 'HAVEPER', 'LIKEFEL', 'ALLBCAR', 'ALLCALC', 'ALLFOL', 'ALLIRON', 'ALLARE', 'ALLB1', 'ALLB12', 'ALLB6', 'ALLVITC', 'ALLVITD', 'ALLVITE', 'ALLZINC', 'ALLB2', 'NOLIKE', 'EXPECT', 'DNTKNOW', 'DONTKNO', 'MENODEP', 'IMEDTHR', 'EMBDDEV', 'E2AVE', 'ESTRDA1', 'ESTRDA2', 'ESTRNJ1', 'ESTRNJ2', 'ESTROG1', 'ESTROG2', 'EFPDFIB', 'EFPB1', 'EFPB12', 'EFPB6', 'EFPARE', 'EFP', 'LEKDISC', 'EXERCIS', 'EXERGEN', 'EXERHAR', 'EXERMEM', 'EXERMEN', 'EXEROST', 'EXEROTH', 'EXERSPE', 'EXERADV', 'EXERPER', 'EXERWGH', 'EXERLOO', 'EXER12H', 'FACEI1', 'FACEI10', 'FACEI11', 'FACEI12', 'FACEI13', 'FACEI14', 'FACEI15'

### ⚖ Normalizing variable names to find common variables

Given that the column names have in their name the visit id at the end, we have to take it away to proceed more easily

In [5]:
"""
    Normalize the variable names.

    This requires getting rid of the last character in each column name, which, as stated
    above, is the visit ID, so we can concatenate all the dataframes together.

    Note: not all the dataframes have the same columns, so we need to get the union of all
    the columns in all the dataframes.
"""

visit_dfs = [*get_visit_records()]

C:\Users\Administrador\AppData\Local\Temp\ipykernel_396\3460416426.py:18: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
C:\Users\Administrador\AppData\Local\Temp\ipykernel_396\3460416426.py:18: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
C:\Users\Administrador\AppData\Local\Temp\ipykernel_396\3460416426.py:18: DtypeWarning: Columns (3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,51,52,53,54,57,58,59,60,61,63,64,65,66,67,68,69,70,71,72,73,74,75,85,87,88,90,91,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,477,478,482,483,484,485,486,490,491,495,504,506,507,515,522,530,536,537,538,539,577) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
C:\Users\Administrador\AppData\

In [6]:
visit_dfs[0].head()

,SWANID,VISIT,INTDAY1,AGE1,LANGINT1,RACE,PREGNAN1,PREVBLO1,EATDRIN1,STRTPER1,...,SPSCDAY1,SPSCTIM1,SPSCMOD1,HPSCDAY1,HPSCTIM1,HPSCMOD1,SPBMDT1,HPBMDT1,BMDFLG1,STATUS1
0,10046,1,413,53,1,2,1,1,1,2,...,413,0:10:08,5,413,0:09:52,5,1.1277,.9619,0,7
1,10056,1,357,52,1,4,1,,1,2,...,441,0:12:55,11,441,0:13:03,11,.914,.8499,0,4
2,10092,1,364,46,1,4,1,,1,1,...,364,0:18:01,5,364,0:17:39,5,1.0377,.8412,0,4
3,10126,1,442,50,1,1,1,,1,2,...,,.,,,.,,,,,4
4,10153,1,374,52,1,3,1,,1,2,...,402,0:10:47,11,402,0:10:40,11,1.0673,1.0313,0,4


In [7]:
def normalize_columns(df, suffix: str):
    # Remove last digit (matching index + 1) at the end, if it exists
    return [col.removesuffix(suffix) if col.endswith(suffix) else col for col in df.columns]


unique_variables = set(normalize_columns(visit_dfs[0], '1'))


for index, visit_df in enumerate(visit_dfs[1:]):

    index_str = str(index + 2)  # Index starts at 0, but we continue at 2
    columns_no_visit = normalize_columns(visit_df, index_str)
    unique_variables &= set(columns_no_visit)

variable_inter = list(unique_variables)
len(variable_inter)

291

In [8]:
variable_inter

['HAPPY',
 'OTHTW11',
 'DISLIKE',
 'OTC3',
 'BROKEBO',
 'HIPMEAS',
 'THYRTW1',
 'SPBMDT',
 'BCPTWI1',
 'THYRREM',
 'BPTW2',
 'HIP',
 'SITESPE',
 'COMBTW2',
 'COMBTW1',
 'RACE',
 'FERTIL1',
 'CHOLTW2',
 'STOPOTH',
 'WAIST',
 'HARTTW1',
 'OTHMED1',
 'BRSTFEE',
 'OTHMED7',
 'SERIPRO',
 'EATDRIN',
 'SPSCMOD',
 'SYSBP2',
 'LANGSAA',
 'PAINTW1',
 'DIZZY',
 'LEGALPR',
 'GOOD',
 'HOSPSTA',
 'FORGET',
 'NUMDAND',
 'BLEEDNG',
 'OTHME13',
 'NOLIKE',
 'BCP2',
 'HYSTERE',
 'SYSBP1',
 'IMPRMEM',
 'VISIT',
 'OTCTW4',
 'EINJTW2',
 'ESTRDA1',
 'HEART1',
 'OSTEOAR',
 'SWANID',
 'HOPEFUL',
 'PAINTW2',
 'EXPENSI',
 'DIABP2',
 'AVCIGDA',
 'WAKEUP',
 'STIFF',
 'PILING',
 'INSULN1',
 'OTHMED2',
 'STATUS',
 'COMBIN1',
 'ANGINA',
 'APPETIT',
 'DONTKNO',
 'UTERPRO',
 'CHOLST2',
 'OTHRTW6',
 'HOURSPA',
 'BCP1',
 'HAVEPER',
 'YOUNGLK',
 'QLTYLIF',
 'CHILDMO',
 'CANCER',
 'MAJEVEN',
 'BLUES',
 'PULSE',
 'BPTW1',
 'NITESWE',
 'ESTRNJ1',
 'CHOLST1',
 'NERVTW1',
 'DANDC',
 'MONEYPR',
 'HAPPEN1',
 'BP1',
 'OTHTW14',
 

In [9]:
# Having variables shared across all the dataframes, we can now filter them out by
# our list of selected variables

selected_shared_variables = set(variables) & set(variable_inter)
len(selected_shared_variables)

49

In [10]:
selected_shared_variables

{'ADVISTO',
 'AGE',
 'ALCHL24',
 'AVCIGDA',
 'BONES1',
 'BONES2',
 'BONES3',
 'BROKEBO',
 'CANCER',
 'COMBIN1',
 'COMBIN2',
 'DIABETE',
 'DNTKNOW',
 'DONTKNO',
 'E2AVE',
 'ESTRDA1',
 'ESTRDA2',
 'ESTRNJ1',
 'ESTRNJ2',
 'ESTROG1',
 'ESTROG2',
 'EXPENSI',
 'FRNADVI',
 'HAVEPER',
 'HCPADVI',
 'HORMOTH',
 'HOTFLAS',
 'HOURSPA',
 'IMPRMEM',
 'LANGINT',
 'LANGSAA',
 'LIKEFEL',
 'MENOSYM',
 'NOLIKE',
 'NOREASO',
 'NOREMEB',
 'OSTEOPO',
 'OSTEOPR',
 'OUTCOME',
 'PHYSILL',
 'PRBBLEE',
 'PRGNANT',
 'RACE',
 'REDUHAR',
 'REGPERI',
 'SIDEEFF',
 'STOPOTH',
 'VAGINDR',
 'YOUNGLK'}

### 📝 Renaming columns and final dataframe

In [11]:
for index, visit_df in enumerate(visit_dfs):

    normalized_columns = normalize_columns(visit_df, str(index + 1))

    # Check each column name in each dataframe, and see if it's in the
    # list of selected variables
    for col, normal_col in zip(visit_df.columns, normalized_columns):

        columns_to_rename = {}

        if normal_col in selected_shared_variables:
            columns_to_rename[col] = normal_col

        # Substitute the column name with the normalized one
        visit_dfs[index].rename(columns=columns_to_rename, inplace=True)


In [12]:
# Check that the columns have been renamed correctly
visit_dfs[0][list(selected_shared_variables)].head()

,NOREASO,PRBBLEE,ESTRNJ1,BROKEBO,DNTKNOW,BONES3,ESTROG1,HORMOTH,HCPADVI,RACE,...,REGPERI,BONES1,E2AVE,OSTEOPR,YOUNGLK,DIABETE,ESTRNJ2,CANCER,VAGINDR,STOPOTH
0,-1,-1,1,0,-1,-1,2,2,1,2,...,1,-1,70.15,1,1,1,,-1,1,-1
1,-1,-1,1,0,-1,-1,1,-1,-1,4,...,-1,-1,194.55,1,-1,1,,-1,1,-1
2,-1,-1,1,1,-1,-1,1,-1,-1,4,...,-1,PINKY FINGER BONE BROKE,250.65,1,-1,1,,-1,1,-1
3,-1,-1,1,0,-1,-1,1,-1,-1,1,...,-1,-1,187.95,1,-1,1,,-1,2,-1
4,-1,-1,1,0,-1,-1,1,-1,-1,3,...,-1,-1,119.5,1,-1,1,,-1,1,-1


In [13]:
visit_dfs[1][list(selected_shared_variables)]

,NOREASO,PRBBLEE,ESTRNJ1,BROKEBO,DNTKNOW,BONES3,ESTROG1,HORMOTH,HCPADVI,RACE,...,REGPERI,BONES1,E2AVE,OSTEOPR,YOUNGLK,DIABETE,ESTRNJ2,CANCER,VAGINDR,STOPOTH
0,,,1,0,,,2,2,2,2,...,1,,33.1,1,1,1,,,1,
1,,,1,0,,,1,,,4,...,,,45.05,1,,1,,,1,
2,,,1,0,,,1,,,1,...,,,17.9,1,,1,,,2,
3,,,1,0,,,1,,,3,...,,,28.25,1,,1,,,1,
4,,,1,0,,,1,,,2,...,,,57.1,1,,1,,,1,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2743,,,1,0,,,1,2,2,4,...,1,,,1,1,1,,,2,
2744,,,1,0,,,1,,,3,...,,,26.75,1,,1,,,1,
2745,,,1,0,,,1,,,4,...,,,370.75,1,,1,,,1,
2746,,,1,0,,,1,,,2,...,,,302.2000000001,1,,1,,,1,


### 🔗 Concatenating dataframes

In [14]:
# Concatenate all the dataframes together
# TODO: check a way to preserve empty values
df = pd.concat((df[list(selected_shared_variables)] for df in visit_dfs))

# Write it to a file
df.to_csv('./datasets/visit_dfs.csv', index=False)

# ❌ Removing columns with several missing values

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25487 entries, 0 to 2244
Data columns (total 49 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   NOREASO  25487 non-null  object
 1   PRBBLEE  25487 non-null  object
 2   ESTRNJ1  25487 non-null  object
 3   BROKEBO  25487 non-null  object
 4   DNTKNOW  25487 non-null  object
 5   BONES3   25487 non-null  object
 6   ESTROG1  25487 non-null  object
 7   HORMOTH  25487 non-null  object
 8   HCPADVI  25487 non-null  object
 9   RACE     25487 non-null  int64 
 10  FRNADVI  25487 non-null  object
 11  PHYSILL  25487 non-null  object
 12  SIDEEFF  25487 non-null  object
 13  OUTCOME  25487 non-null  object
 14  LANGSAA  25487 non-null  object
 15  ALCHL24  25487 non-null  object
 16  AGE      25487 non-null  object
 17  HOTFLAS  25487 non-null  object
 18  ADVISTO  25487 non-null  object
 19  LANGINT  25487 non-null  object
 20  PRGNANT  25487 non-null  object
 21  REDUHAR  25487 non-null  object
 22 

# 📆 Adding exact date of visit